#  赛题十三 —— 论文复现自验报告

赛题内容：在 MindQuantum 框架下用元学习解决组合优化问题

论文题目：A Quantum Approximate Optimization Algorithm with Metalearning for MaxCut Problem and Its Simulation via TensorFlow Quantum
 
复现要求：利用 MindQuantum 实现 MetaQAOA 算法，复现 fig7 中的量子线路，并得到 fig 的结论。

## 赛题介绍

量子近似优化算法（Quantum Approximate Optimization Algorithm，QAOA）是利用量子计算机来近似解决组合优化问题的量子算法，其可以多项式时间复杂度来求解组合优化问题。

在本论文采用元学习算法对量子变分网络的参数进行优化：即利用经典的 long short-term memory（LSTM）作为经典优化器，来更新量子变分网络的参数，来求解一个典型的组合优化问题：最大割问题（Max-Cut）。

关于 Max-Cut 问题及如何用 QAOA 算法求解，在 MindQuantum 官网的教程案例中已有详细介绍可供参考：[量子近似优化算法](https://www.mindspore.cn/mindquantum/docs/zh-CN/master/quantum_approximate_optimization_algorithm.html)
此处不再赘述，而仅对论文核心部分进行介绍。

元学习是指学习如何设计机器学习模型，从而使其学习地更好、更快地学习。一个典型的案例是用一个神经网络（A）去学习如何优化另一个神经网络（B）的参数。此时，神经网络（A）的作用其实就等同于一个外部优化器。
LSTM 是一种特殊的循环神经网络，其能够避免长时期依赖的问题，从而能够记住很长时间内的信息。关于 LSTM 的详细解释和在 MindSpore 中的使用方法，请参见 [LSTM](https://www.mindspore.cn/docs/zh-CN/r1.7/api_python/nn/mindspore.nn.LSTM.html?highlight=LSTM)。

本论文的方案是以 LSTM 作为量子神经网络的外部优化器。

量子线路的参数表示为 $\{\overrightarrow\eta, \overrightarrow\gamma\}$，而 LSTM 中的参数表示为 $\{\overrightarrow\varphi\}$。

其算法流程为：

0. 随机初始化 LSTM 参数 $\{\overrightarrow{\varphi_0}\}$ 并预测一组量子线路的参数 $\{\overrightarrow\eta, \overrightarrow\gamma\}$。
1. 在量子线路中执行参数 $\{\overrightarrow\eta, \overrightarrow\gamma\}$ 并得到哈密顿量的期望值。
2. 以哈密顿量的期望值作为损失值，更新 LSTM 的参数 $\{\overrightarrow\varphi\}$。
3. LSTM 预测一组新的量子线路的参数 $\{\overrightarrow\eta, \overrightarrow\gamma\}$。
- 重复上述过程步骤 1. - 3. ，直到收敛为止。

该流程展现在下图中：

![lstm_and_qnn](./src/lstm_and_qnn.png)

对于不同节点的 Max-Cut 问题，我们范例性考虑 3-regular 情况，即每个节点上有 3 条边。并基于 MindQuantum 对 6 和 8 节点 Max-Cut 的问题，利用 QAOA 算法进行求解。

## 自验结果

通过运行 `6_nodes.ipynb` 和 `8_nodes.ipynb` 中的程序，可知，对于不同节点的 Max-Cut 问题，采用 QAOA 算法得到的结果和经典算法得到的结果一致，这确保了正确性。

下图展示了 6 节点 Max-Cut 问题求解，期望的割边数随训练次数的变化情况。可见，采用 LSTM 网络作为外部优化器，可有效优化量子网络的参数。
（该图通过运行 `6_nodes.ipynb` 中的 `主程序 1` 得到，结论和论文图 8 的结果一致）。

![6_nodes.png](./src/6_nodes.png)

下图展示了 8 节点 Max-Cut 问题求解，期望的割边数随训练次数的变化情况。可见，采用 LSTM 网络作为外部优化器，可有效优化量子网络的参数。
（该图通过运行 `8_nodes.ipynb` 中的 `主程序 1` 得到，结论和论文图 8 的结果一致）。

![8_nodes.png](./src/8_nodes.png)

**结论：** 通过以上复现结果，可见作为原学习算法的重要分支，以 LSTM 网络作为量子线路的外部优化器，确实可以起到很好的优化作用。


## 自验环境

硬件：华为云 CouldIDE、 2U8G 20G 存储 、Linux 系统

软件：Mindspore 1.6.0、Mindquantum 0.5.0rc1、numpy 1.21.5

训练超参数：learning rate = 0.1、optimizer = Adam、max_step = 500
